# Save T-T cross-correlations for a single column
#### 0. Choose vertical levels for interpolation
#### 1. Interpolate temperature to chosen vertical levels
#### 2. Compute T-T cross-correlations for a single column

## Load packages and data

### Load packages

In [1]:
import numpy as np
import xarray as xr

### Load data

In [2]:
path_to_repo = '/Users/zstanley/Documents/git_repos/mlcdc' ## change this 
my_data_dir = path_to_repo + '/data'

In [3]:
columns = xr.open_dataset(my_data_dir+'/tropical_pacific_columns.nc')

## Compute T-T cross-correlations
#### Compute correlations without interpolating, remember to subtract out the column mean

In [4]:
column_means = columns.mean(dim=['ens_mem'])

/Users/zstanley/.conda/envs/favorite/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Users/zstanley/.conda/envs/favorite/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [5]:
column_perturbations = columns - column_means

In [6]:
corr = xr.corr(column_perturbations['atm_T'], column_perturbations['ocn_Temp'], dim=['ens_mem', 'lat', 'lon'])

In [618]:
cov = xr.cov(column_perturbations['atm_T'], column_perturbations['ocn_Temp'], dim=['ens_mem', 'lat', 'lon'])

In [7]:
print('The correlation between SST and AST is: ' +str(corr.sel(atm_lev=126, ocn_lev=1).values))

The correlation between SST and AST is: 0.3980252359600756


In [619]:
print('The covariance between SST and AST is: ' +str(cov.sel(atm_lev=126, ocn_lev=1).values))

The covariance between SST and AST is: 0.003478526830553949


## Choose vertical levels for interpolation

In [8]:
ocn_z = columns['ocn_h'].mean(['ens_mem', 'lat', 'lon']).cumsum(dim='ocn_lev')
atm_p = columns['atm_delp'].mean(['ens_mem', 'lat', 'lon']).cumsum(dim='atm_lev') / 1e2

In [9]:
## What is atm_DZ?
columns['atm_DZ']

<xarray.DataArray 'atm_DZ' (ens_mem: 80, atm_lev: 127, lat: 5, lon: 5)>
array([[[[-2702.0977  , ..., -2711.1926  ],
         ...,
         [-2709.0774  , ..., -2711.9258  ]],

        ...,

        [[  -21.948128, ...,   -21.953274],
         ...,
         [  -21.868338, ...,   -21.865795]]],


       ...,


       [[[-2698.812   , ..., -2701.7158  ],
         ...,
         [-2698.8188  , ..., -2695.3408  ]],

        ...,

        [[  -21.942108, ...,   -21.972502],
         ...,
         [  -21.853186, ...,   -21.866037]]]], dtype=float32)
Coordinates:
  * lon      (lon) float32 207.5 208.5 209.5 210.5 211.5
  * lat      (lat) float32 10.5 11.5 12.5 13.5 14.5
  * atm_lev  (atm_lev) float32 0.0 1.0 2.0 3.0 4.0 ... 123.0 124.0 125.0 126.0
  * ens_mem  (ens_mem) int64 0 1 2 3 4 5 6 7 8 9 ... 71 72 73 74 75 76 77 78 79
Attributes:
    long_name:        DZ
    units:            unknown
    _CoordinateAxes:  atm_lev lat lon